## Objective: To extract the text of the English language letters included in the North American Immigrant Letters, Diaries and Oral Histories collection.

Source: /Users/alaynemoody/Dropbox/Thesis/North_American_Letters_Diaries_OralHistories/dataNAIL

Search Criteria: "Letter" in column "doctype" and "English" in column "language" in IMLD_DOCS_QA completed.xlsx

Results: 1556 results

Steps:

<ol>
<li>For first row,</li>
<ol>
    <li>Open the sourceid</li>
    <li>Extract contents of the "part id" tag with the docid.</li>
    <li>Save text to a file.</li></ol>

<li>Use a loop to do this for a subset of sample</li>
<li>Run loop for all rows in sample</li>
</ol>    

In [110]:
# Import libraries
import pandas as pd 
from bs4 import BeautifulSoup
import nltk
import re

In [2]:
# 1. Open IMLD_DOCS_QA completed.xlsx
df = pd.read_csv("20201005_AM_FinalMeta.csv") 
df

,Unnamed: 0,docsequence,docid,docyear,doctype,allsubject,broadsubj,personalevent,wwritten,docauthorid,...,stayed_north_america.x,stayed_north_america.y,agewriting,marriagestatus,maternalstatus,author_generation,authorgender,cultural_heritage,religion,north_american_occupation
0,1,23,S10003-D023,1836.0,Letter,Childbirth; Church attendance; Cities; Farms; ...,Health; Religion; Communities; Relationships; ...,NaN,"Baltimore, MD; Maryland; United States; Mid-At...",per0022938,...,Stayed,No,23.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
1,2,24,S10003-D024,1836.0,Letter,Church services; Farming; Letters and mail; Lo...,Religion; Life Styles; Politics; Communities; ...,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,23.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
2,3,25,S10003-D025,1837.0,Letter,Correspondence; Crops; English language; Livin...,Intellectual life; Agriculture; Domestic life;...,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,24.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
3,4,26,S10003-D026,1837.0,Letter,American Indians; Construction; Fevers; Floods...,Ethnic groups; Economics; Health; Environment;...,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,24.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
4,5,27,S10003-D027,1838.0,Letter,Brothers; Deaths; Farms; Fires; Letters and ma...,Relationships; Health; Communities; Environmen...,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,25.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
5,6,28,S10003-D028,1838.0,Letter,Aunts; Family arguments; Fevers; Parenting; Si...,Relationships; Health,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,25.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
6,7,29,S10003-D029,1838.0,Letter,Brothers; Fevers; Livestock; Militias; Neighbo...,Relationships; Health; Agriculture; Military,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,25.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
7,8,30,S10003-D030,1839.0,Letter,Childbirth; Daughters; Immigration and emigrat...,Health; Relationships; Domestic life; Communities,NaN,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,26.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
8,9,31,S10003-D031,1839.0,Letter,Alcoholism; Brothers; Cousins; Fevers; Rain; H...,Health; Relationships; Environment,Physical illness of sibling,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,26.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician
9,10,32,S10003-D032,1839.0,Letter,Alcoholism; Construction; Correspondence; Cous...,Health; Economics; Intellectual life; Relation...,Physical illness of other family member,"Westphalia, MO; Missouri; United States; West ...",per0022938,...,Stayed,No,26.0,Married,Mother,First,F,German; European,Catholic; Christian,Homemaker; Physician


In [282]:
df[['docid', 'docauthorid']].describe()[0:2]

,docid,docauthorid
count,1222,1222
unique,1222,285


In [303]:
# Display key criteria for following set of instructions. 
df[['docid']].head()

,docid
0,S10003-D023
1,S10003-D024
2,S10003-D025
3,S10003-D026
4,S10003-D027


In [4]:
# Select the values in docid and split into sourceid (1st element) and docid (2nd element)
IDs = df["docid"].str.split(pat = "-", expand=True)
IDs["2"] = df["docid"]
IDs.columns = ['Src', 'Doc', 'Full']
IDs.head()

,Src,Doc,Full
0,S10003,D023,S10003-D023
1,S10003,D024,S10003-D024
2,S10003,D025,S10003-D025
3,S10003,D026,S10003-D026
4,S10003,D027,S10003-D027


In [310]:
IDs.iloc[136,2]

'S1019-D002'

## 1a. Open the sourceid

In [132]:
# Open the first source in the list
f = open("dataNAIL/" + IDs.iloc[0,0] + ".txt","r", encoding = 'utf-8')
print(f)

<_io.TextIOWrapper name='dataNAIL/S10003.txt' mode='r' encoding='utf-8'>


In [133]:
# Parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(f, 'html.parser')

# View the HTML
print(soup.prettify())

<!DOCTYPE LAD SYSTEM "LAD-TEI.DTD">
<lad>
 <header>
  <author n="A22938">
   <source n="S10003"/>
  </author>
 </header>
 <text>
  <page n="NA">
   <part id="S10003-D001">
    <div1 name="Frontmatter" type="frontmatter">
     <div2 name="Title Page and Credits" type="titlepage">
      <p align="r">
       <b>
        hold
dear,
as always
       </b>
      </p>
      <p>
       Edited by Adolf E&amp;dot Schroeder
and Carla Schulz-Geisberg
      </p>
      <p>
       Translated from German by Adolf E&amp;dot Schroeder
      </p>
      <p>
       Jette, a German Immigrant Life in Letters
      </p>
      <p>
       University of Missouri Press
      </p>
      <p>
       Columbia, 1988
      </p>
      <page n="NA">
       <p>
        Copyright © 1988 by
       </p>
       <p>
        The Curators of the University of Missouri
       </p>
       <p>
        University of Missouri Press, Columbia, Missouri 65211
       </p>
       <p>
        Printed and bound in the United States of Ameri

## 1b. Extract elements of the part tag

In [134]:
# Isolate the desired part (i.e., doc) and place in a variable
doc = soup.find(id=IDs.iloc[0,2])

# View the part
print(doc)

<part id="S10003-D023">
<p align="r">
Baltimore
</p>
<p align="r">
20 September 1836
</p>
<p>
Dear Heinrich:
</p>
<p>
Friday evening, 16 September, anchor was dropped and we had safely arrived in the
harbor! It was a long and arduous voyage! However we were and are all well and thank the
Father in Heaven that he has protected us so far! I had written down the events of the voyage
for you, but to my great annoyance I am now missing the whole notebook; perhaps I will find
it later.<note ref="93">93 The notebook is not known to have survived.</note> As you know, we set sail on 12 July. By the afternoon, when the pilot left us, one after
the other began to hold his head overboard; Bernhard, Mrs&amp;dot Schwarze, the children, and
others. The next morning I felt ill and remained in bed for two days, but during the whole
time I only had to vomit four or five times; this was pretty much the same with most passengers.
Later I was always spared.
</p>
<p>
The wind was adverse from the very first

In [135]:
doc = ''.join(text for text in doc.find_all(text=True) if text.parent.name != "note")
print(doc)



Baltimore


20 September 1836


Dear Heinrich:


Friday evening, 16 September, anchor was dropped and we had safely arrived in the
harbor! It was a long and arduous voyage! However we were and are all well and thank the
Father in Heaven that he has protected us so far! I had written down the events of the voyage
for you, but to my great annoyance I am now missing the whole notebook; perhaps I will find
it later. As you know, we set sail on 12 July. By the afternoon, when the pilot left us, one after
the other began to hold his head overboard; Bernhard, Mrs&dot Schwarze, the children, and
others. The next morning I felt ill and remained in bed for two days, but during the whole
time I only had to vomit four or five times; this was pretty much the same with most passengers.
Later I was always spared.


The wind was adverse from the very first evening and remained so for almost three weeks;
we did not pass the English Channel until 2 August and had to cope with very heavy stormy
weather

Part of the author's autobiography made it into this document. I manually erased this from the file and searched the other output files to ensure that this error did not present elsewhere.

## 1c. Save string to file

In [210]:
f = open("letters/" + IDs.iloc[0,2], "w")
f.write(doc)
f.close()

## 2. Use a loop to do the above for a sample

In [129]:
sampleIDs = IDs.sample(n=3)
sampleIDs

,Src,Doc,Full
419,S12296,D010,S12296-D010
968,S9831,D059,S9831-D059
1001,S9841,D029,S9841-D029


In [289]:
sampleIDs.iloc[1,2]

'S11067-D038'

In [130]:
for index in range(len(sampleIDs)):
    f = open("dataNAIL/" + sampleIDs['Src'].iloc[index] + ".txt","r", encoding = 'utf-8')
    soup = BeautifulSoup(f, 'html.parser')
    doc = soup.find(id=sampleIDs.iloc[index,2])
    doc = ''.join(text for text in doc.find_all(text=True) if text.parent.name != "note")
    f = open("letters/" + sampleIDs.iloc[index,2] + ".txt", "w")
    f.write(doc)
    f.close()
print("done")

done


## 3. Run loop for all rows in sample

In [131]:
for index in range(len(IDs)):
    f = open("dataNAIL/" + IDs['Src'].iloc[index] + ".txt","r", encoding = 'utf-8')
    soup = BeautifulSoup(f, 'html.parser')
    doc = soup.find(id=IDs.iloc[index,2])
    doc = ''.join(text for text in doc.find_all(text=True) if text.parent.name != "note")
    f = open("letters/" + IDs.iloc[index,2] + ".txt", "w")
    f.write(doc)
    f.close()
print("done")

done
